<a href="https://colab.research.google.com/github/Jkanishkha0305/AI-Agents/blob/main/Introspective_Agent_Financial_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### How to Use Introspective Agent with Reflection in Financial Tasks?

This Agent will perform financial tasks while utilizing the reflection agent pattern.

This agent is Based on Research work [CRITIC: LARGE LANGUAGE MODELS CAN SELF- CORRECT WITH TOOL-INTERACTIVE CRITIQUING](https://arxiv.org/pdf/2305.11738)

👉 This introspective agent has been implemented in LlamaIndex .   

👉 In the introspective agents, there are 2 agents:

▪ The Agent Worker : This first agent will generate an initial response to the task (which is the user query), setting the stage for further analysis. (if this agent is not provided, the user query will be considered as a first answer)

▪Then the Critic Agent will perform reflection and correction on the generated ouput from the agent worker. This process will be performed in a loop until a stopping condition is met or a maximum number of iterations is reached.


👉 I used only one tool : "get historical prices for a given ticker".


👉 I tested several type of agents:

▪ With the Tool Interactive Reflection Agent:
 - When specifying or not the use of an Agent Worker
 - When specifying or not the tool in the Agent Worker

▪ When using the Agent Worker only

▪ When using the Self-Reflection Agent


### Introspective Agent Worker using LLamaIndex for Financial Tasks

1. Install Libraries

In [ ]:
%%capture
!pip install llama-index-agent-introspective
!pip install llama-index-llms-openai
!pip install llama-index-program-openai
!pip install llama-index-readers-file

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 12.9 MB/s eta 0:00:00


2. Specifying Tools: Get Historical Prices

In [ ]:
from llama_index.core.tools.tool_spec.base import BaseToolSpec
import yfinance as yf
import pandas as pd
from datetime import date
from datetime import datetime


class FinanceTools(BaseToolSpec):
    """Finance tools spec."""
    #Only one tool example for this notebook tutorial
    spec_functions = [
        "stock_prices",
    ]
    def __init__(self) -> None:
      """Initialize the Yahoo Finance tool spec."""

    def stock_prices(self, ticker: str, start_date : str, end_date = datetime.today().strftime('%Y-%m-%d')) -> pd.DataFrame:
        """
        Get the historical prices and volume for a ticker from start_date to end_date.
            Args:
              ticker (str): the stock ticker to be given to yfinance
              start_date (str): the start date in the format YYYY-MM-DD
              end_date (str): the end date in the format YYYY-MM-DD
        """

        stock = yf.Ticker(ticker)
        df = yf.download(ticker, start=start_date, end=end_date)
        return df


In [ ]:
finance_tool = FinanceTools()
finance_tool_list = finance_tool.to_tool_list()

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

Source from LlamaIndex: https://docs.llamaindex.ai/en/latest/examples/agent/introspective_agent_toxicity_red

3. Introspective Agent with Tool Interactive Reflection

In [ ]:
from llama_index.agent.introspective import ToolInteractiveReflectionAgentWorker, IntrospectiveAgentWorker
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgentWorker
from llama_index.core.llms import ChatMessage, MessageRole


def get_introspective_agent_with_tool_interactive_reflection( verbose=True, with_main_worker=True):
    """Helper function for building introspective agent using tool-interactive␣ ↪reflection.
    Steps:
    1. Define the `ToolInteractiveReflectionAgentWorker`
    1a. Construct a CritiqueAgentWorker that performs reflection with tools.
    1b. Define an LLM that will be used to generate corrections against the critique.
    1c. Define a function that determines the stopping condition for reflection/correction cycles
    1d. Construct `ToolInteractiveReflectionAgentWorker` using from_defaults()
    2. Optionally define a `MainAgentWorker`
    3. Construct `IntrospectiveAgent`
        3a. Construct `IntrospectiveAgentWorker` using .from_defaults()
        3b. Construct `IntrospectiveAgent` using .as_agent()
    """

    # 1a.
    critique_agent_worker = FunctionCallingAgentWorker.from_tools(
        tools=finance_tool_list, llm=OpenAI("gpt-3.5-turbo"), verbose=verbose
    )

    # 1b.
    correction_llm = OpenAI("gpt-4-turbo-preview")

    # 1c.
    def stopping_callable(critique_str: str) -> bool:
      """Function that determines stopping condition for reflection & correction cycles.
      critique_str [str]: The response string provided by the critique agent.
      """
      return "[PASS]" in critique_str

    # 1d.
    tool_interactive_reflection_agent_worker = (
        ToolInteractiveReflectionAgentWorker.from_defaults(
            critique_agent_worker=critique_agent_worker,
            critique_template=(
                "Retrieve the exact interval times of historical prices as requested by the user."
                "Check the code carefully for correctness, style, and efficiency, and give constructive criticism for how to improve it. "
                "write '[PASS]' otherwise write '[FAIL]'. "
                "Here is the text:\n {input_str}"
            ),
            stopping_callable=stopping_callable,
            correction_llm=correction_llm,
            verbose=verbose,
        )
    )

    # 2.
    if with_main_worker:
        main_agent_worker = OpenAIAgentWorker.from_tools(
            tools=finance_tool_list, llm=OpenAI("gpt-4-turbo-preview"),verbose=True
            )
    else:
      main_agent_worker = None

    # 3a.
    introspective_agent_worker = IntrospectiveAgentWorker.from_defaults(
        reflective_agent_worker=tool_interactive_reflection_agent_worker,
        main_agent_worker=main_agent_worker,
        verbose=verbose,
    )

    chat_history = [
            ChatMessage(
                content="You are a financial assistant that helps answering questions to gather historical prices and propose Python implementation of rading strategies.",
                role=MessageRole.SYSTEM,
            )
    ]

    # 3b.
    return introspective_agent_worker.as_agent(
        chat_history=chat_history, verbose=verbose
    )

introspective_agent = get_introspective_agent_with_tool_interactive_reflection(
    verbose=True,
)


3.1 Using Agent Worker : with_main_worker = True

In [ ]:
# Without sepcifying the agent worker TOOLS:
        # main_agent_worker = OpenAIAgentWorker.from_tools(
        # tools=[], llm=OpenAI("gpt-4-turbo-preview"), verbose=True
        #)

query="""Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices."""
response = await introspective_agent.achat(query)
#The critic agent didn't use the Tools to answer the question, instead, it proposes a Python code to fetch data.

> Running step a5b3c5c1-cc3c-4b28-a55c-513ceb1c06db. Step input: Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices.
> Running step 61b6cc56-421c-466a-8d8f-39c7ff1e7713. Step input: Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices.
Added user message to memory: Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices.


[*********************100%%**********************]  1 of 1 completed

=== Calling Function ===
Calling function: stock_prices with args: {"ticker":"BTC-USD","start_date":"2023-09-01"}
Got output:                     Open          High           Low         Close  \
Date                                                                 
2023-09-01  25934.021484  26125.869141  25362.609375  25800.724609   
2023-09-02  25800.910156  25970.285156  25753.093750  25868.798828   
2023-09-03  25869.472656  26087.148438  25817.031250  25969.566406   
2023-09-04  25968.169922  26081.525391  25657.025391  25812.416016   
2023-09-05  25814.957031  25858.375000  25589.988281  25779.982422   
...                  ...           ...           ...           ...   
2024-05-08  62332.640625  62986.085938  60877.128906  61187.941406   
2024-05-09  61191.199219  63404.914062  60648.074219  63049.960938   
2024-05-10  63055.191406  63446.742188  60208.781250  60792.777344   
2024-05-11  60793.355469  61451.152344  60492.625000  60793.710938   
2024-05-12  60793.503906  61818.15

> Running step 15822ef3-2109-453b-b48b-c5c2fdfc3e37. Step input: ```plaintext
                 Close
Date                  
2023-09-01  25800.724609
2023-09-02  25868.798828
2023-09-03  25969.566406
2023-09-04  25812.416016
2023-09-05  25779.982422
...                 ...
2024-05-08  61187.941406
2024-05-09  63049.960938
2024-05-10  60792.777344
2024-05-11  60793.710938
2024-05-12  61448.394531

[255 rows x 1 columns]
```
> Running step f35a499c-37e7-4171-8f88-e23ad818bbd3. Step input: Retrieve the exact interval times of historical prices as requested by the user.Check the code carefully for correctness, style, and efficiency, and give constructive criticism for how to improve it. write '[PASS]' otherwise write '[FAIL]'. Here is the text:
 ```plaintext
                 Close
Date                  
2023-09-01  25800.724609
2023-09-02  25868.798828
2023-09-03  25969.566406
2023-09-04  25812.416016
2023-09-05  25779.982422
...                 ...
2024-05-08  61187.941406
2024-05-09  6304

In [ ]:
# By sepcifying the agent worker TOOLS:
        # main_agent_worker = OpenAIAgentWorker.from_tools(
        # tools=finance_tool_list, llm=OpenAI("gpt-4-turbo-preview"), verbose=True
        #)

query="""Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices."""
response = await introspective_agent.achat(query)

#The first attempt of the agent worker was not correct as it gives historical prices starting 01 of September.
# The critic agent didn't succeed either: Fail to detect the right ticker symbol (even if the agent worker succeed).
# However it proposes a Python code to fetch data (with the right ticker).

> Running step 269ed392-a497-493e-ac49-8e2d0ce72bda. Step input: Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices.
> Running step 334624c8-835c-4c30-88b6-6551bfc38a16. Step input: Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices.
Added user message to memory: Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices.
> Running step 84196161-5219-4314-af4f-1fbeaf959cfc. Step input:                  Close
Date                  
2023-09-01  25800.724609
2023-09-02  25868.798828
2023-09-03  25969.566406
2023-09-04  25812.416016
2023-09-05  25779.982422
...                 ...
2023-11-26  16502.796875
2023-11-27  16468.849609
2023-11-28  16394.324219
2023-11-29  16801.371094
2023-11-30  17052.074219

[91 rows x 1 columns]
> Running step ab7f4a6a-dd8d-4f67-a901-9e7a8bf9558a. Step input: Retrieve the exact 

3.2 Not Using Agent Worker : with_main_worker=False

In [ ]:
query="""Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices."""
response = await introspective_agent.achat(query)
#The final anwser is good

> Running step c309b7a2-b58b-4bbe-a56c-f6ea20b5a314. Step input: Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices.
> Running step d6fea91c-7827-4149-9621-7f2e2cf49b02. Step input: Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices.
Added user message to memory: Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices.
> Running step 6d4e5360-eb34-4965-bc02-7d5969a5a65e. Step input: ```plaintext
                 Close
Date                  
2023-09-01  25800.724609
2023-09-02  25868.798828
2023-09-03  25969.566406
2023-09-04  25812.416016
2023-09-05  25779.982422
...                 ...
2023-11-26  16542.349609
2023-11-27  16402.378906
2023-11-28  16422.730469
2023-11-29  16801.849609
2023-11-30  17052.183594
```
> Running step f3465653-7c6e-454a-a03c-e36680c3b8a6. Step input: Retrieve the exact interv

In [ ]:
list_resp = response.sources

In [ ]:
list_resp[0].raw_output

IndexError: list index out of range

4. When Using Only One Agent Worker with Tools: Without In- trospective Agent

In [ ]:
main_agent_worker = OpenAIAgentWorker.from_tools(
            tools=finance_tool_list, llm=OpenAI("gpt-4-turbo-preview"),verbose=True
        )

chat_history = [
        ChatMessage(
            content="You are a financial assistant that helps answering questions to gather historical prices and propose Python implementation of trading strategies.",
            role=MessageRole.SYSTEM,
        )
]

agent_worker = main_agent_worker.as_agent(
    chat_history=chat_history, verbose=True
)

query="""Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices."""
response3 = await agent_worker.achat(query)

> Running step 2a58667b-5151-442a-a2e8-18f4dce3924d. Step input: Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices.
Added user message to memory: Give me the last 3 months historical close prices of BTCUSD. Respond only with the dataframe with the close prices.


[*********************100%%**********************]  1 of 1 completed

=== Calling Function ===
Calling function: stock_prices with args: {"ticker":"BTC-USD","start_date":"2023-09-13"}
Got output:                     Open          High           Low         Close  \
Date                                                                 
2023-09-13  25837.554688  26376.113281  25781.123047  26228.324219   
2023-09-14  26228.277344  26774.623047  26171.451172  26539.673828   
2023-09-15  26533.818359  26840.498047  26240.701172  26608.693359   
2023-09-16  26606.199219  26754.769531  26473.890625  26568.281250   
2023-09-17  26567.927734  26617.998047  26445.074219  26534.187500   
...                  ...           ...           ...           ...   
2024-05-08  62332.640625  62986.085938  60877.128906  61187.941406   
2024-05-09  61191.199219  63404.914062  60648.074219  63049.960938   
2024-05-10  63055.191406  63446.742188  60208.781250  60792.777344   
2024-05-11  60793.355469  61451.152344  60492.625000  60793.710938   
2024-05-12  60793.503906  61818.15

5. With Self Reflection agent:

The intuition here is that if you don’t specify any tool (as in the example notebook from LlamaIn- dex), you will not have any answer ==> However, let’s sepcify the tools in the agent worker and see if the self reflection agent will help:

In [ ]:
from llama_index.agent.introspective import SelfReflectionAgentWorker

def get_introspective_agent_with_self_reflection(
    verbose=True, with_main_worker=True
):
    """Helper function for building introspective agent using self reflection.
    Steps:
    1. Define the `SelfReflectionAgentWorker`
      1a. Construct `SelfReflectionAgentWorker` using .from_defaults()
    2. Optionally define a `MainAgentWorker`
    3. Construct `IntrospectiveAgent`
      3a. Construct `IntrospectiveAgentWorker` using .from_defaults()
      3b. Construct `IntrospectiveAgent` using .as_agent()
    """

    # 1a.
    self_reflection_agent_worker = SelfReflectionAgentWorker.from_defaults(
          llm=OpenAI("gpt-4-turbo-preview"),
          verbose=verbose,
    )

    # 2.
    if with_main_worker:
        main_agent_worker = OpenAIAgentWorker.from_tools(
              tools=finance_tool_list, llm=OpenAI("gpt-4-turbo-preview"),verbose=True
        )
    else:
        main_agent_worker = None

    # 3a.
    introspective_worker_agent = IntrospectiveAgentWorker.from_defaults(
        reflective_agent_worker=self_reflection_agent_worker,
        main_agent_worker=main_agent_worker,
        verbose=verbose,
    )

    chat_history = [
        ChatMessage(
            content="You are a financial assistant that helps answering questions to gather historical prices and propose Python implementation of trading strategies.",
            role=MessageRole.SYSTEM,
        )
    ]

    # 3b.
    return introspective_worker_agent.as_agent(
        chat_history=chat_history, verbose=verbose
    )

introspective_agent_self_reflection = get_introspective_agent_with_self_reflection(
    verbose=True
)

In [ ]:
query="""Give me the last 3 months historical close prices of BTCUSD. Respond␣ ↪only with the dataframe with the close prices."""
response2 = await introspective_agent_self_reflection.achat(query)

> Running step fa819732-eb28-4824-9494-9a10473f931f. Step input: Give me the last 3 months historical close prices of BTCUSD. Respond␣ ↪only with the dataframe with the close prices.
> Running step b08f5a42-d980-4399-af04-575489a6b4de. Step input: Give me the last 3 months historical close prices of BTCUSD. Respond␣ ↪only with the dataframe with the close prices.
Added user message to memory: Give me the last 3 months historical close prices of BTCUSD. Respond␣ ↪only with the dataframe with the close prices.


[*********************100%%**********************]  1 of 1 completed

=== Calling Function ===
Calling function: stock_prices with args: {"ticker":"BTC-USD","start_date":"2023-09-01"}
Got output:                     Open          High           Low         Close  \
Date                                                                 
2023-09-01  25934.021484  26125.869141  25362.609375  25800.724609   
2023-09-02  25800.910156  25970.285156  25753.093750  25868.798828   
2023-09-03  25869.472656  26087.148438  25817.031250  25969.566406   
2023-09-04  25968.169922  26081.525391  25657.025391  25812.416016   
2023-09-05  25814.957031  25858.375000  25589.988281  25779.982422   
...                  ...           ...           ...           ...   
2024-05-08  62332.640625  62986.085938  60877.128906  61187.941406   
2024-05-09  61191.199219  63404.914062  60648.074219  63049.960938   
2024-05-10  63055.191406  63446.742188  60208.781250  60792.777344   
2024-05-11  60793.355469  61451.152344  60492.625000  60793.710938   
2024-05-12  60793.503906  61818.15

> Running step 039bd0d2-95f8-4bd1-a49c-8b07fc5ccc9c. Step input: ```plaintext
                 Close
Date                  
2023-09-01  25800.724609
2023-09-02  25868.798828
2023-09-03  25969.566406
2023-09-04  25812.416016
2023-09-05  25779.982422
...                 ...
2024-05-08  61187.941406
2024-05-09  63049.960938
2024-05-10  60792.777344
2024-05-11  60793.710938
2024-05-12  61448.394531
```
> Reflection: {'is_done': False, 'feedback': "The tool call arguments incorrectly specify the start date without an end date, leading to data retrieval beyond the requested last 3 months. Additionally, the final assistant message only presents the data without any accompanying explanation or confirmation that it addresses the user's request for the last 3 months of historical close prices for BTCUSD. The assistant should have specified an end date corresponding to the last date of data retrieval and provided a summary or confirmation that the data presented covers the requested time frame."}